In [2]:
from dataprocess.kdd_cup99 import KDD_CUP_99_DataLoader
from dataprocess.unsw_nb15 import UNSW_NB15_DataLoader
from net.CNN import CNN_2_linear, CNN
# from net.multCNN import multCNN

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [4]:
import os 
import time

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

t = time.localtime()
year, month, day = t.tm_year, t.tm_mon, t.tm_mday

net = input('input the name of net to train: ')
mode = input('input the tag about the training: ')
if mode == '':
    mode = 'no_tag'

log_directory = f"/Transfer-learning-IDS/log/exp/{net}/{mode}/{month}_{day}/"
ckpt_directory = f"/Transfer-learning-IDS/history/{net}/{mode}/checkpoint/{month}_{day}/"
if not os.path.isdir(log_directory):
    os.makedirs(log_directory)
if not os.path.isdir(ckpt_directory):
    os.makedirs(ckpt_directory)
print('log_directory: ', log_directory)
print('ckpt_directory: ', ckpt_directory)


cuda
log_directory:  /Transfer-learning-IDS/log/exp/CNN/KDD_CUP99_Adam/5_6/
ckpt_directory:  /Transfer-learning-IDS/history/CNN/KDD_CUP99_Adam/checkpoint/5_6/


In [5]:
# for kdd_cup99
dataset = KDD_CUP_99_DataLoader('E:/DataSets/kddcup.data', 256)
data_labels = dataset.data.to_num_column_dic[41].keys()
class_num = len(data_labels)
discrete_column_idx = dataset.data.discrete_column
discrete_column_idx.remove(41)
continuous_column_idx = [i for i in range(41) if i not in discrete_column_idx]
input_channel_kind = len(discrete_column_idx)
input_channel_num = len(continuous_column_idx)

print(discrete_column_idx, continuous_column_idx, data_labels)

[1, 2, 3, 11, 13, 14, 20, 21] [0, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40] dict_keys([b'smurf.', b'neptune.', b'normal.', b'satan.', b'ipsweep.', b'portsweep.', b'nmap.', b'back.', b'warezclient.', b'teardrop.', b'pod.', b'guess_passwd.', b'buffer_overflow.', b'land.', b'warezmaster.', b'imap.', b'rootkit.', b'loadmodule.', b'ftp_write.', b'multihop.', b'phf.', b'perl.', b'spy.'])


In [6]:
# # for UNSW_NB15
# dataset = UNSW_NB15_DataLoader('E:/DataSets/UNSW-NB15 - CSV Files', 256)
# data_labels = dataset.data.label_dic.keys()
# class_num = len(data_labels)
# discrete_column_idx = dataset.data.discrete_column
# discrete_column_idx.remove(47)
# discrete_column_idx.remove(48)
# continuous_column_idx = [i for i in range(47) if i not in discrete_column_idx]
# input_channel_kind = len(discrete_column_idx)
# input_channel_num = len(continuous_column_idx)

# print(discrete_column_idx, continuous_column_idx, data_labels)

In [7]:
# net = CNN_2_linear(input_channel_num + input_channel_kind, class_num).to(device)
net = CNN(input_channel_num + input_channel_kind, class_num).to(device)
epoch = 100
criterion = nn.CrossEntropyLoss().to(device)
# optimizer = optim.SGD(net.parameters(), lr=1e-1)
optimizer = optim.Adam(net.parameters(), lr=1e-1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.25)  

In [8]:
writer1 = SummaryWriter(log_directory)

In [9]:
# ckpt = torch.load('/Transfer-learning-IDS/history/CNN_KDD/checkpoint/ckpt_best_99.pth')
# epoch_start = ckpt["epoch"]
# net.load_state_dict(ckpt["net"])
# optimizer.load_state_dict(ckpt["optimizer"])

In [10]:
for t in range(0, epoch):
    running_loss = 0
    for step, (x, y) in enumerate(dataset):
        
        x = x.to(device)
        y = F.one_hot(y.long(), num_classes=class_num).float()
        y = y.to(device)
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()

        y_pred = net(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        
        # visualize loss
        running_loss += loss.item()
    # ...log the running loss
    writer1.add_scalar('training loss', running_loss, t)
    print("epoch: %d, loss: %f, learning rate: %f" % (t, running_loss,optimizer.param_groups[0]['lr']))
    scheduler.step()
    if t % 10 == 9:
        checkpoint = {"net": net.state_dict(), 'optimizer':optimizer.state_dict(), "epoch": t}
        torch.save(checkpoint,  ckpt_directory + 'ckpt_best_%s.pth' %(str(t)))
writer1.close()

epoch: 0, loss: 30241.624328, learning rate: 0.100000
epoch: 1, loss: 30240.290520, learning rate: 0.100000
epoch: 2, loss: 30240.274872, learning rate: 0.100000
epoch: 3, loss: 30240.306146, learning rate: 0.100000
epoch: 4, loss: 30240.313920, learning rate: 0.100000
epoch: 5, loss: 30240.282713, learning rate: 0.100000
epoch: 6, loss: 30240.310043, learning rate: 0.100000
epoch: 7, loss: 30240.313943, learning rate: 0.100000
epoch: 8, loss: 30240.274879, learning rate: 0.100000
epoch: 9, loss: 30240.282699, learning rate: 0.100000
epoch: 10, loss: 30240.317845, learning rate: 0.100000
epoch: 11, loss: 30240.302243, learning rate: 0.100000
epoch: 12, loss: 30240.313990, learning rate: 0.100000
epoch: 13, loss: 30240.267077, learning rate: 0.100000
epoch: 14, loss: 30240.313945, learning rate: 0.100000
epoch: 15, loss: 30240.302212, learning rate: 0.100000
epoch: 16, loss: 30240.290509, learning rate: 0.100000
epoch: 17, loss: 30240.329556, learning rate: 0.100000
epoch: 18, loss: 302

KeyboardInterrupt: 